In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

import sklearn.manifold as skm 
import sklearn.utils.random as skr # for random sampling 


In [3]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    # Early stopping 
    epsilon = 0.001
    counter = 0
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
        if max(test_acc) - test_acc[-1] >= epsilon:
            counter += 1
            #if counter >= 20:
            #    sys.stderr.write("Early stopping initiated")
            #    print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            #    break
        else:
            counter = 0
    return np.asarray(train_acc), np.asarray(test_acc)

In [4]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename+".pkl", "wb"), compress=True)
    file = open(filename+".txt", "a")
    msg = str(100*max(train_acc)) + " " + str(100*max(test_acc)) + " " +\
        str(len(train_acc)) + " " + str(np.argmax(test_acc) + 1) + "\n"
    file.write(msg)
    file.close()

In [5]:
############### ISOMAP ########################
# Everyone uses a neighborhood size of 8-12.
# If your manifold is more than 2-3 dimensional, then Isomap probably won't work, 
# you would need to have a neighborhood size that is larger (more points!) so that 
# you have reasonable options to estimate your geodesic path with links to nearby neighbors.
def iso_wrapper(n_neighbors, n_components):
    
    start = time.time()
    isomap = skm.Isomap(n_neighbors, n_components)
    isomap.fit(manifolddata)
    end = time.time()
    # Preparation time 
    prep_time = int(end - start) 
    print( "Prep time: ", prep_time ) 
    
    start = time.time()
    trainiso = isomap.transform(traindata)
    testiso = isomap.transform(testdata)
    end = time.time() 
    # Transformation time
    trans_time = int(end-start)
    print( "Transform time: ", trans_time )

    filename = "./dumper/isomap_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainiso, trainlabels, testiso, testlabels), open(filename, "wb"), compress=True)
    
    return trainiso, testiso, prep_time, trans_time

In [6]:
############### Modified/Hessian Locally Linear Embedding , more ########################

def lle_wrapper(n_neighbors, n_components, lle_type = "modified"):
    
    start = time.time()
    lle = skm.LocallyLinearEmbedding(n_neighbors, n_components, method = lle_type)
    lle.fit(manifolddata)
    end = time.time()
    prep_time = int(end - start) 
    print( prep_time ) 

    start = time.time()
    trainlle = lle.transform(traindata)
    testlle = lle.transform(testdata)
    end = time.time()
    trans_time = int(end-start)
    print( trans_time )

    filename = "./dumper/"+lle_type+"_data_"+str(n_neighbors)+"_"+str(n_components)+".pkl"
    joblib.dump((trainlle, trainlabels, testlle, testlabels), open(filename, "wb"), compress=True)

    return trainlle, testlle, prep_time, trans_time

In [7]:
############### Spectral Embedding ########################
############### Multi-dimensional Scaling ########################
############### t-distributed Stochastic Neighbor Embedding #######
# No online learning version; has to input full data

In [8]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset UCIHAR from UCIHAR
Loading train data... train data of shape (6213, 561) loaded
Loading test data...  test  data of shape (1554, 561) loaded
Data Loaded. Num of features = 561 Num of Classes = 12

In [9]:
# data cropping
traindata = traindata[:25000]
testdata = testdata[:5000]

In [10]:
print(trainlabels)

[5 4 4 ... 4 5 5]


In [11]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'UCIHAR', 'dataset': 'UCIHAR', 'D': 2500, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 5, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 250, 'nFeatures': 561, 'nClasses': 12}


In [12]:
# Sample code for running manifolding learning data
# Random sampling for manifold 

# manifoldlist = skr.sample_without_replacement(len(traindata), 2000)
# manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])
# n_neighbors = 200
# n_components  = 25
# param["nFeatures"] = n_components
# param["manifold"] = "modified"
# traintrans, testtrans, prep_time, trans_time = mlle_wrapper(n_neighbors, n_components)

In [13]:
# Sample code for running basiline data 
# param["nFeatures"] = nFeatures
# param["manifold"] = "baseline"
# traintrans, testtrans = traindata, testdata

In [14]:
################# VANILLA #################
def vanilla_train(param):
    hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
    basis = hdb.getBasis()
    bid = hdb.getParam()["id"]
    # Update param with bid
    param = hdb.getParam()
    print(bid)

    hde = HDE.HD_encoder(basis)
    trainencoded = hde.encodeData(traintrans)
    #trainencoded = hde.encodeData(traindata)
    HDE.saveEncoded(trainencoded, trainlabels, bid, "train")
    testencoded = hde.encodeData(testtrans)
    #testencoded = hde.encodeData(testdata)
    HDE.saveEncoded(testencoded, testlabels, bid, "test")

    train_accs = []
    test_accs = []
    # Should have 95%
    for i in range(param["iter_per_encoding"]):
        hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
        train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels, param)
        train_accs.append(train_acc)
        test_accs.append(test_acc)
    
    return train_accs, test_accs


In [15]:
## Pamam needed to be specified: D, nFeatures, n_neighbors, n_components, "manifold"

# Best MLLE param for arbitrary D on MNIST is currently (100,30)
n_neighbors = 125
n_components  = 40
maniset_size = 200   # Size of the manifold set

for n_neighbors  in [125]:

    # Random sampling for manifold 
    manifoldlist = skr.sample_without_replacement(len(traindata), maniset_size)
    manifolddata = np.asfarray(traindata[np.asarray(manifoldlist)])

    param["nFeatures"] = nFeatures
    #param["nFeatures"] = n_components
    
    param["manifold"] = "baseline"
    #param["manifold"] = "modified"
      
    traintrans, testtrans, prep_time, trans_time = traindata, testdata, 0, 0
    #traintrans, testtrans, prep_time, trans_time = lle_wrapper(n_neighbors, n_components, "modified")

    for dim in [100, 200, 300, 400]:
        param["D"] = dim
        train_accs, test_accs = vanilla_train(param)
        #filename = "./logfile/" + param["manifold"] + "_" +str(param["D"])+"_"+str(param["id"])
        filename = "./logfile/" + param["manifold"] + "_" +str(param["D"])+"_"+ \
            str(n_neighbors) + "_" + \
            str(n_components) + "_" + \
            str(param["id"])
        file = open(filename+"_time.txt","w")
        file.write("Preparation time: "+str(prep_time)+" Transform time: "+str(trans_time)+"\n")
        file.close()

        for i in range(len(train_accs)):
            dump_log(param, train_accs[i], test_accs[i], filename)


Generating vanilla HD basis of shape... 


6036


(100, 561)
Encoding time: 0.027463436126708984 
Dumping basis into file: base_6036.pkl 
Encoding data of shape (6213, 561)


Time spent: 3 sec
Dumping data into encoded_6036_train.pkl 
Encoding data of shape (1554, 561)


Time spent: 0 sec
Dumping data into encoded_6036_test.pkl 
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


...

Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.920972 	 	 Test: 0.923423
Train: 0.935941 	 	 Test: 0.942728
Train: 0.951875 	 	 Test: 0.904118
Train: 0.952358 	 	 Test: 0.935006
Train: 0.961693 	 	 Test: 0.951094
Train: 0.964268 	 	 Test: 0.952381
Train: 0.967488 	 	 Test: 0.953024
Train: 0.971028 	 	 Test: 0.960746
Train: 0.971350 	 	 Test: 0.949807
Train: 0.967327 	 	 Test: 0.957529
Train: 0.970063 	 	 Test: 0.953024
Train: 0.973926 	 	 Test: 0.947876
Train: 0.974569 	 	 Test: 0.946589
Train: 0.974569 	 	 Test: 0.963964
Train: 0.971189 	 	 Test: 0.963320
Train: 0.974730 	 	 Test: 0.961390
Train: 0.975213 	 	 Test: 0.936293
Train: 0.971511 	 	 Test: 0.954955
Train: 0.978754 	 	 Test: 0.955598
Train: 0.973765 	 	 Test: 0.956885
Train: 0.974891 	 	 Test: 0.955598
Train: 0.975857 	 	 Test: 0.944659
Train: 0.979881 	 	 Test: 0.936937
Train: 0.974408 	 	 Test: 0.921493
Train: 0.981651 	 	 Test: 0.961390
Train: 0.979076 	 	 Test: 0.958172
Train: 0.978110 	 	 Test: 0.962677
Train: 0.976340 	 	 Test: 0.960746
Train: 0.977145 	 	 

Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.913890 	 	 Test: 0.914414
Train: 0.937067 	 	 Test: 0.922136
Train: 0.952519 	 	 Test: 0.943372
Train: 0.955899 	 	 Test: 0.946589
Train: 0.960567 	 	 Test: 0.949163
Train: 0.965717 	 	 Test: 0.956242
Train: 0.965556 	 	 Test: 0.959459
Train: 0.968131 	 	 Test: 0.956885
Train: 0.966844 	 	 Test: 0.954955
Train: 0.968131 	 	 Test: 0.931145
Train: 0.971833 	 	 Test: 0.958816
Train: 0.970868 	 	 Test: 0.962677
Train: 0.975696 	 	 Test: 0.958816
Train: 0.968775 	 	 Test: 0.962033
Train: 0.974730 	 	 Test: 0.953668
Train: 0.975857 	 	 Test: 0.936293
Train: 0.976018 	 	 Test: 0.964607
Train: 0.977145 	 	 Test: 0.958172
Train: 0.979237 	 	 Test: 0.953668
Train: 0.976179 	 	 Test: 0.962677
Train: 0.975213 	 	 Test: 0.964607
Train: 0.973121 	 	 Test: 0.906049
Train: 0.977628 	 	 Test: 0.924710
Train: 0.978271 	 	 Test: 0.954311
Train: 0.979398 	 	 Test: 0.957529
Train: 0.976179 	 	 Test: 0.963964
Train: 0.977306 	 	 Test: 0.966538
Train: 0.977949 	 	 Test: 0.949807
Train: 0.977949 	 	 

KeyboardInterrupt: 